In [5]:
import requests
import time
import jwt
import sqlite3
import pandas as pd


def generate_jwt_token(client_id, client_secret):
    now = int(time.time())
    payload = {
        "iss": client_id,
        "sub": client_id,
        "iat": now,
        "exp": now + 3600,  # Token expires in 1 hour
    }
    jwt_token = jwt.encode(payload, client_secret, algorithm='HS256')
    return jwt_token


def get_access_token(client_id, client_secret):
    jwt_token = generate_jwt_token(client_id, client_secret)

    token_endpoint = "https://rest.arbeitsagentur.de/oauth/gettoken_cc"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    response = requests.post(token_endpoint, headers=headers, data=data)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token. Status code: {response.status_code}")


def fetch_entgelte_data(token, kldb_schluessel, params):
    headers = {
        "Authorization": f"Bearer {token}"
    }
    url = f"https://rest.arbeitsagentur.de/infosysbub/entgeltatlas/pc/v1/entgelte/{kldb_schluessel}"
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data


def create_table(db_connection):
    cursor = db_connection.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS entgelte (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        kldb TEXT,
                        region_id INTEGER,
                        region_bezeichnung TEXT,
                        region_schluessel TEXT,
                        region_oberRegionId TEXT,
                        region_oberRegionBezeichnung TEXT,
                        region_beitragsBemessungsGrenze INTEGER,
                        gender_id INTEGER,
                        gender_bezeichnung TEXT,
                        ageCategory_id INTEGER,
                        ageCategory_bezeichnung TEXT,
                        performanceLevel_id INTEGER,
                        performanceLevel_bezeichnung TEXT,
                        branche_id INTEGER,
                        branche_bezeichnung TEXT,
                        entgelt INTEGER,
                        entgeltQ25 INTEGER,
                        entgeltQ75 INTEGER,
                        besetzung INTEGER
                    )''')


def insert_data_into_db(data, db_connection):
    cursor = db_connection.cursor()
    for entry in data:
        if all(value is not None for value in entry.values()):
            values = (
                entry["kldb"],
                entry["region"]["id"],
                entry["region"]["bezeichnung"],
                entry["region"]["schluessel"],
                entry["region"]["oberRegionId"],
                entry["region"]["oberRegionBezeichnung"],
                entry["region"]["beitragsBemessungsGrenze"],
                entry["gender"]["id"],
                entry["gender"]["bezeichnung"],
                entry["ageCategory"]["id"],
                entry["ageCategory"]["bezeichnung"],
                entry["performanceLevel"]["id"],
                entry["performanceLevel"]["bezeichnung"],
                entry["branche"]["id"],
                entry["branche"]["bezeichnung"],
                entry["entgelt"],
                entry["entgeltQ25"],
                entry["entgeltQ75"],
                entry["besetzung"]
            )

            cursor.execute('''INSERT INTO entgelte (
                kldb,
                region_id,
                region_bezeichnung,
                region_schluessel,
                region_oberRegionId,
                region_oberRegionBezeichnung,
                region_beitragsBemessungsGrenze,
                gender_id,
                gender_bezeichnung,
                ageCategory_id,
                ageCategory_bezeichnung,
                performanceLevel_id,
                performanceLevel_bezeichnung,
                branche_id,
                branche_bezeichnung,
                entgelt,
                entgeltQ25,
                entgeltQ75,
                besetzung
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', values)

    db_connection.commit()


def main():
    client_id = "c4f0d292-9d0f-4763-87dd-d3f9e78fb006"
    client_secret = "566c4dd6-942f-4cda-aad6-8d611c577107"

    # Read KldB-Schlüssel values from Excel file into a DataFrame
    df = pd.read_excel('kldb.xlsx')
    kldb_schluessel_list = df['kldb_id'].tolist()

    # Get the access token
    token = get_access_token(client_id, client_secret)

    # Fetch entgelte data
    params = {
        "r": "1"
    }

    # Create SQLite database connection
    db_connection = sqlite3.connect('weiterbildung_analysis.db')

    # Create table if not exists
    create_table(db_connection)
    for kd in kldb_schluessel_list:
        try:
            # Insert data into the database
            entgelte_data = fetch_entgelte_data(token, kd, params)
            insert_data_into_db(entgelte_data, db_connection)

            print(f"Data fetched and saved to SQLite database for KldB-Schlüssel: {kd}")

        except Exception as e:
            print(f"Error: {e}")
            db_connection.rollback()  # Rollback changes in case of an error

    db_connection.close()


if __name__ == "__main__":
    main()


Data fetched and saved to SQLite database for KldB-Schlüssel: 84304
Data fetched and saved to SQLite database for KldB-Schlüssel: 11102
Data fetched and saved to SQLite database for KldB-Schlüssel: 11103
Data fetched and saved to SQLite database for KldB-Schlüssel: 11104
Data fetched and saved to SQLite database for KldB-Schlüssel: 1111
Data fetched and saved to SQLite database for KldB-Schlüssel: 11113
Data fetched and saved to SQLite database for KldB-Schlüssel: 11114
Data fetched and saved to SQLite database for KldB-Schlüssel: 1112
Data fetched and saved to SQLite database for KldB-Schlüssel: 11123
Data fetched and saved to SQLite database for KldB-Schlüssel: 11124
Data fetched and saved to SQLite database for KldB-Schlüssel: 1113
Data fetched and saved to SQLite database for KldB-Schlüssel: 11132
Data fetched and saved to SQLite database for KldB-Schlüssel: 11133
Data fetched and saved to SQLite database for KldB-Schlüssel: 1118
Data fetched and saved to SQLite database for KldB-S